In [93]:
from apyori import apriori, load_transactions
from helper_functions import load_dataset

In [94]:
transactions = None
try:
    data = load_dataset('../data/assignment1_income_levels_cleaned.xlsx')
    transactions = data.astype(str).values.tolist()
except FileNotFoundError:
    print('File not found')

In [95]:
min_support = 0.2
min_confidence = 0.95
min_length = 2
results = list(apriori(transactions, min_support=min_support, min_confidence=min_confidence, min_length=min_length))

file_name = f'../output/output_{min_support}_{min_confidence}_{min_length}.txt'
try:
    with open(file_name, 'w') as file:
        for rule in results:
            if len(rule.items) == 1:
                continue
            file.write(f"Items: {', '.join(rule.items)}\n")
            for i in range(0, len(rule.ordered_statistics)):
                antecedents = ', '.join(rule.ordered_statistics[i].items_base)
                consequents = ', '.join(rule.ordered_statistics[i].items_add)
                
                if len(antecedents) == 0 or len(consequents) == 0:
                    continue
                    
                support = str(rule.support)[:7]
                confidence = str(rule.ordered_statistics[i].confidence)[:7]
                lift = str(rule.ordered_statistics[i].lift)[:7]
            
                file.write(f"\t{antecedents} => {consequents} (support: {support}, confidence: {confidence}, lift: {lift})")
            file.write('\n')
            
except FileNotFoundError:
    print('File not found')